## 2)

In [1]:
from ortools.linear_solver import pywraplp as glp   

# a)

In [2]:
mymodel = glp.Solver('TF', glp.Solver.GLOP_LINEAR_PROGRAMMING)

# if it works it works
Wheat = mymodel.NumVar(0, mymodel.infinity(), 'Wheat')
WheatConstraint = mymodel.Constraint(600, mymodel.infinity())
WheatConstraint.SetCoefficient(Wheat, 1) 

Corn = mymodel.NumVar(0, mymodel.infinity(), 'Corn')
CornConstraint = mymodel.Constraint(0, 1000)
CornConstraint.SetCoefficient(Corn, 1) 

Soybeans = mymodel.NumVar(0, mymodel.infinity(), 'Soybeans')
SoybeansConstraint = mymodel.Constraint(0, 1200)
SoybeansConstraint.SetCoefficient(Soybeans, 1) 

Barley = mymodel.NumVar(0, mymodel.infinity(), 'Barley') # these are all in tons

# $/acre divided by tons/acre gives units of $/acre, same units as planting cost
Profit = mymodel.Objective()
Profit.SetCoefficient(Wheat, 170 - (150/2.0))
Profit.SetCoefficient(Corn, 150 - (135/2.5))
Profit.SetCoefficient(Soybeans, 140 - (140/3.0))
Profit.SetCoefficient(Barley, 190 - (150/1.5)) 
Profit.SetMaximization()

# the variables are in tons so need to take inverse of production to get units of acre/ton, ton cancels giving us units of acres
LandConstraint = mymodel.Constraint(0, 1000)
LandConstraint.SetCoefficient(Wheat, 1/2) 
LandConstraint.SetCoefficient(Corn, 1/2.5)
LandConstraint.SetCoefficient(Soybeans, 1/3)
LandConstraint.SetCoefficient(Barley, 1/1.5)

In [3]:
mymodel.Solve()

print('Optimal Solution:')
print('Total Profit = %.2f' % Profit.Value())

print('Wheat Quantity (tons) = %.2f' % Wheat.solution_value())
print('Corn Quantity (tons) = %.2f' % Corn.solution_value())
print('Soybeans Quantity (tons) = %.2f' % Soybeans.solution_value())
print('Barley Quantity (tons) = %.2f' % Barley.solution_value())

# marginal value of the resources
print('Land Constraint dual value = %.2f' % LandConstraint.dual_value())
print('Wheat Constraint dual value = %.2f' % WheatConstraint.dual_value())
print('Corn Constraint dual value = %.2f' % CornConstraint.dual_value())
print('Soybeans Constraint dual value = %.2f' % SoybeansConstraint.dual_value())


Optimal Solution:
Total Profit = 241000.00
Wheat Quantity (tons) = 600.00
Corn Quantity (tons) = 750.00
Soybeans Quantity (tons) = 1200.00
Barley Quantity (tons) = 0.00
Land Constraint dual value = 240.00
Wheat Constraint dual value = -25.00
Corn Constraint dual value = 0.00
Soybeans Constraint dual value = 13.33


# b)
Yes, because the marginal value per acre of land is 240, which is greater than the cost of 100\\$. Profit will increase by 140\\$

# c)
They should try and produce less wheat since the dual value is negative, which means the quota is reducing profits. They should also try and increase the cap on soybeans - it has a positive dual value and profit would increase if they could produce more